In [ ]:
import pulp

In [2]:

import csv
import pandas as pd
import numpy as np
import itertools

End of file name for data to be fed to model

In [ ]:
# fname = 'ideal' # first choice of each student uniquely ranked 1

fname = '0'     # randomly generated seelction form

In [3]:
# build student project selection form by importing
with open(f'data/student_preference_matrices/student_preference_matrix_{fname}.csv', 'r') as f:   
        
        cost_matrix = csv.reader(f)
        
        cost_matrix = [[int(i) for i in j] for j in cost_matrix] # convert from string
        
        cost_matrix = np.array(cost_matrix)
        
print(cost_matrix)
        
n_students, n_projects = cost_matrix.shape

[[1 0 0 0 0 0 0 0 0 4 3 2 5 0 0]
 [0 1 0 0 4 0 0 0 5 0 3 2 0 0 0]
 [0 0 1 0 2 0 0 0 3 4 0 0 0 5 0]
 [0 0 0 1 0 4 0 0 2 0 0 0 0 3 5]
 [0 0 0 0 1 5 0 0 2 0 0 0 4 0 3]
 [0 0 0 0 0 1 4 0 0 5 0 3 0 0 2]
 [0 0 0 4 0 0 1 0 3 0 0 0 5 0 2]
 [0 0 4 0 0 0 0 1 0 0 0 2 3 0 5]
 [0 0 0 0 0 0 0 2 1 4 0 3 5 0 0]
 [0 0 0 0 3 0 0 0 0 1 0 5 2 4 0]]


Student grades weighting, $\alpha$ 

(set all weights as equal if not using this property)

In [4]:
alpha = np.ones(n_students)

print(alpha)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Max students each project can take, $\chi$ (i.e. project capacity vector)

In [5]:
# chi = np.ones(n_projects) 

# chi[-2:] = 2

# chi[-1] = 3

with open(f'data/student_preference_matrices/project_capacity_{fname}.csv', 'r') as f:
    capacity = list(csv.reader(f))
    capacity = [int(i) for i in capacity[0]]
    chi = capacity

print(chi)

[1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1]


Students who must be garunteed a project, $\omega$

In [155]:
omega = np.ones(n_students)

print(omega)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Create a linear programming maximization problem

In [156]:
prob = pulp.LpProblem("Project_matching", pulp.LpMaximize)

Create a dictionary of items (cartesian product of students and projects i.e. tuple for each combination of student i and porject j) with:
- category `continuous` i.e. the optimization solution can take any real-numbered value greater than zero.

In [157]:
x = pulp.LpVariable.dicts("x",                                  # name
                          itertools.product(range(n_students),  # list of all possible student-project combinations 
                                            range(n_projects)),
                          cat=pulp.LpBinary                     # binary output data
                         )  

Main objective function (i.e. function to maximize by setting values of $x_{ij}$ for each $ij$)

${maximize} \sum_{i=1}^{I} \sum_{j=1}^{J} x_{ij} \frac{\alpha_{i}}{A_{ij}}$

Where:

$\alpha_{i}$= grade, student $i$ 
<br> $x_{i}$= project selected $i \in [0,1]$ 
<br> $A_{ij}$= element of project selection matrix $A$

In [158]:
objective_function = 0                  # default value of solution is 0 (for all unselected projects)

for student in range(n_students):
    for project in range(n_projects):
        if cost_matrix[(student, project)] > 0:  # if project selected (and ranked) by student
            objective_function += x[(student, project)] * (alpha[student]) / ((cost_matrix[(student, project)]))
                
    prob += objective_function

Constraints

In [159]:
# Students can only be allocated projects they have chosen
for student, project in x:
    prob += x[student, project] <= float(cost_matrix[student, project])
    
# Max students per project = chi_j students for every project j
for project in range(n_projects):
    prob += sum(x[(student, project)] for student in range(n_students)) <= chi[project]
    
# At most 1 project per student
for student in range(n_students):
    prob += sum(x[(student, project)] for project in range(n_projects)) <= 1

# At least omega_i project per student
for student in range(n_students):
    prob += sum(x[(student, project)] for project in range(n_projects)) >= omega[student]

Find solution

In [160]:
prob.solve()

print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


Extract project assignments

In [6]:
studentID, projectID = [], []               # store student project assignments

assign_matrix = np.zeros(cost_matrix.shape) # create an array to display assignments

for v in prob.variables():
    
    if v.varValue>0:    # choose all values where x_ij = 1
        
#         print(v.name, "=", v.varValue)
#         print(type(v.name), "=", v.varValue)

        # process the name string to get the student and project number 
        punc = '''x()_'''
        name = v.name
        for p in name:
            if p in punc:
                name = name.replace(p, '')
        name = name.split(',')
        name = [int(n) for n in name]

        assign_matrix[name[0], name[1]] = 1
        
        studentID.append(name[0])
        projectID.append(name[1])

NameError: name 'prob' is not defined

Check every student has a project

In [ ]:
print('\ntotal_projects_assigned=',
      np.sum(assign_matrix), 
      '\nequal to number of students=', 
      np.sum(assign_matrix)==n_students)

Display outcome

In [162]:
assign_matrix *= cost_matrix                                     # matrix showing cost of each assignment   

ranks = assign_matrix[np.nonzero(assign_matrix)]

print('\nassign matrix\n',np.matrix(np.absolute(assign_matrix)))


print('student', studentID)
print('project allocation', projectID)
print('rank of allocated project', ranks)
print('\n\nsum of all assignments (should be as low as possible)=', ranks.sum())


assign matrix
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]

students [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
project ID (this model) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
project rank [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


sum of all assignments (should be as low as possible)= 10.0
